In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#read General sales file
Target_Sales = pd.read_csv("Target_Sales_2017-2018.csv", encoding = "ISO-8859-1")

dropInd=Target_Sales[Target_Sales['UPC']=='Filler'].index
Target_Sales.drop(index = dropInd, inplace=True)

Target_Sales["UPC"] = Target_Sales["UPC"].astype(np.int64)
Target_Sales["Wk End"] = pd.to_datetime(Target_Sales['Wk End'], infer_datetime_format=True)
#Target_Sales["DayofWeek"] = Target_Sales["Wk End"].dt.dayofweek
Target_Sales["YEAR"] = Target_Sales["Wk End"].dt.year
Target_Sales["WeekNum"] = Target_Sales["Wk End"].dt.week
Target_Sales["YEARWeekNum"] = Target_Sales["YEAR"].astype(str)+"-"+Target_Sales["WeekNum"].astype(str)

C:\Users\80160001\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
Target_Sales.columns

Index(['Filter BU 1', 'BU', 'BU (VisualizationFactView)', 'Brand Name',
       'UPC Title', 'Wk End', 'Filter Year', 'Fiscal Year Number',
       'Gross Sales', 'IRI Category (UPCHierarchyView)',
       'IRI SubCategory (UPCHierarchyView)', 'Filter Parent Name',
       'Pepsi Quarter', 'Period Number', 'Shipped Units', 'UPC',
       'UPC (VisualizationFactView)', 'YEAR', 'WeekNum', 'YEARWeekNum'],
      dtype='object')

In [4]:
TSalesLt = Target_Sales[["UPC","Gross Sales","YEAR", "WeekNum", "YEARWeekNum"]].copy()

In [5]:
TSalesLt.head(3)

,UPC,Gross Sales,YEAR,WeekNum,YEARWeekNum
0,28400243490,13.96,2017,45,2017-45
1,876063002035,13.16,2017,42,2017-42
2,876063002806,3.29,2017,51,2017-51


In [6]:
TSalesLt.isnull().sum() #there were some sales were null, fill them

UPC               0
Gross Sales    9630
YEAR              0
WeekNum           0
YEARWeekNum       0
dtype: int64

In [7]:
TSalesLt.fillna(0, inplace = True)
TSalesLt.isnull().sum()

UPC            0
Gross Sales    0
YEAR           0
WeekNum        0
YEARWeekNum    0
dtype: int64

In [8]:
#read campaign data
TargetCamp = pd.read_csv("TGT_PM_Pepsi_Wrap_Up_Report_4.1.19.csv", encoding = "ISO-8859-1")

TargetCamp.rename(columns = {' Spend ': 'Spend'}, inplace=True)
TargetCamp["UPC"] = TargetCamp["UPC"].astype(np.int64)
TargetCamp["Date"] = pd.to_datetime(TargetCamp['Date'], infer_datetime_format=True)
#TargetCamp["DayofWeek"] = TargetCamp["Date"].dt.dayofweek
TargetCamp["YEAR"] = TargetCamp["Date"].dt.year
TargetCamp["WeekNum"] = TargetCamp["Date"].dt.week
TargetCamp["YEARWeekNum"] = TargetCamp["YEAR"].astype(str)+"-"+TargetCamp["WeekNum"].astype(str)
TargetCampLt = TargetCamp.drop(columns=['Account Id', 'Account Name', 'Campaign Id', 'Campaign Name', 'Date',
                        'TCIN Name', 'Impressions', 'Clicks']).copy()

In [9]:
TargetCampLt.head(3)

,UPC,TCIN,Spend,YEAR,WeekNum,YEARWeekNum
0,12000142086,16958332,4,2018,34,2018-34
1,12000142086,16958332,0,2018,34,2018-34
2,12000142086,16958332,10,2018,34,2018-34


In [10]:
#some variables for campaign year and campaign weeks
campYear = 2018
campWeeks = np.sort(TargetCamp["WeekNum"].unique())

In [11]:
print("unique UPCs {}".format(len(TargetCamp.UPC.unique())))
print("unique TCINs {}".format(len(TargetCamp.TCIN.unique())))

unique UPCs 174
unique TCINs 181


In [12]:
## Optional ##
#since there are some UPC with multiple TCINs, we need to merge those spends for the single UPC
UPCtoTCIN = TargetCamp.groupby(["UPC","TCIN"])["Account Id"].count().reset_index().groupby(["UPC"])["TCIN"].count()

In [13]:
#spot check that the 1  UPC to many TCIN cases
UPCtoTCIN[UPCtoTCIN>1]

UPC
492717000000    3
818094000000    5
836093000000    2
Name: TCIN, dtype: int64

In [14]:
UPCtoTCIN[UPCtoTCIN>1].index

Int64Index([492717000000, 818094000000, 836093000000], dtype='int64', name='UPC')

In [15]:
TargetCamp[TargetCamp["UPC"]==818094000000.0]["TCIN"].unique()

array([12958487, 14771376, 15275597, 14771377, 12958668], dtype=int64)

In [16]:
TargetCamp[TargetCamp["TCIN"]==12958487].head(1) #the first two TCINs for UPC 818094000000

,Account Id,Account Name,Campaign Id,Campaign Name,Date,UPC,TCIN,TCIN Name,Impressions,Clicks,Spend,YEAR,WeekNum,YEARWeekNum
265,2121480,PepsiCo - Target,43903,NATL_Brand Direct_PepsiCo AON Search_Flight_Ca...,2018-08-21,818094000000,12958487,Rockstar Sugar Free Double Strength Energy Dri...,160,1,2,2018,34,2018-34


In [17]:
TargetCamp[TargetCamp["TCIN"]==14771376].head(1) #there are similar product but slightly different
## Optional END

,Account Id,Account Name,Campaign Id,Campaign Name,Date,UPC,TCIN,TCIN Name,Impressions,Clicks,Spend,YEAR,WeekNum,YEARWeekNum
337,2121480,PepsiCo - Target,43903,NATL_Brand Direct_PepsiCo AON Search_Flight_Ca...,2018-08-21,818094000000,14771376,Rockstar Pure Zero Silver Ice Energy Drink - 1...,13,0,0,2018,34,2018-34


In [18]:
#roll the Campaign data into weekly so it matches with the time series frequency of Target Sales data
#also the one UPC to many TCIN cases,  those spend will be summed to the single UPC
#TCgroup=TargetCamp.groupby(["TCIN", "UPC", "YEAR", "WeekNum"]).agg({"Impressions": "sum", "Clicks": "sum", "Spend":"sum" })
TCWeekly = TargetCampLt.groupby(["UPC", "YEAR", "WeekNum", "YEARWeekNum"], as_index=False)["Spend"].sum()

In [19]:
len(TCWeekly.UPC.unique()) #should be 181

174

In [20]:
TCWeekly.head(3)#check that it works

,UPC,YEAR,WeekNum,YEARWeekNum,Spend
0,12000002304,2018,35,2018-35,0
1,12000002304,2018,36,2018-36,8
2,12000002304,2018,37,2018-37,5


In [21]:
TCWeekly["Spend"].sum() # should be 44797

44797

In [22]:
campWeeks

array([27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44], dtype=int64)

In [23]:
#make YearWkNumdf
YearWkNum17 = list(map(lambda x: "2017-"+str(x),campWeeks))
YearWkNum18 = list(map(lambda x: "2018-"+str(x),campWeeks))
YearWkNumdf = pd.DataFrame({"YEAR": [2017]*18 + [2018]*18, "WeekNum": list(campWeeks)*2,
                            "YEARWeekNum":YearWkNum17+YearWkNum18})

In [24]:
YearWkNumdf.tail()

,YEAR,WeekNum,YEARWeekNum
31,2018,40,2018-40
32,2018,41,2018-41
33,2018,42,2018-42
34,2018,43,2018-43
35,2018,44,2018-44


In [25]:
YearWkNumdf.dtypes

YEAR            int64
WeekNum         int64
YEARWeekNum    object
dtype: object

In [26]:
YearWkNumdf.shape

(36, 3)

In [27]:
#check that all Target Campaign UPC  is in sales
#there are actually soome Campaign UPCs that are msising from sales UPCs
SalesUPC = set(TSalesLt.UPC.unique())

In [28]:
CampUPC = set(TCWeekly.UPC.unique())

In [29]:
CampUPC - SalesUPC 
#these are the UPCs that are missing from Sales file 
#check if they have large revenue later

{28400154406,
 28400154437,
 28400154550,
 28400154659,
 28400155861,
 28400655927,
 30000018804,
 492717000000,
 493000000000,
 818094000000,
 836093000000}

In [30]:
joinUPC = CampUPC & SalesUPC #there are 163 UPCs in both files
print(type(joinUPC))
print(len(joinUPC))

<class 'set'>
163


In [31]:
#checkpoint copy DF
TCWeeklyold = TCWeekly.copy() #for checkpoint

In [32]:
TCWeekly.groupby("UPC")["YEARWeekNum"].count().min() #some only had 1 week that had promotional spend
#max() is 18, which means that most of them do not have all 18 weeks of promotion spend>0 

1

In [33]:
TSalesLt.shape #

(67168, 5)

In [34]:
len(TSalesLt.UPC.unique()) #TSales has 1124 UPCS, so the result df should have 1124*36 = 40464 rows

1124

In [35]:
1124*36

40464

In [36]:
163*36 #post joining both TSalesSlice and TClight it should have 5868 rows

5868

In [37]:
def mergeYWN(df):
    #each df is of a single UPC df
    df2 = df.merge(YearWkNumdf, how = "right", left_on="YEARWeekNum", right_on="YEARWeekNum")
    return df2

In [38]:
SalesSlicedf = TSalesLt.groupby(["UPC"]).apply(mergeYWN)

In [39]:
SalesSlicedf.shape #check that shape is correct

(40464, 7)

In [40]:
SalesSlicedf[SalesSlicedf["UPC"].isnull()][:10]

UPC  Gross Sales  YEAR_x  WeekNum_x YEARWeekNum  YEAR_y  \
UPC                                                                    
71090098 17  NaN          NaN     NaN        NaN     2017-27    2017   
         18  NaN          NaN     NaN        NaN     2017-28    2017   
         19  NaN          NaN     NaN        NaN     2017-29    2017   
         20  NaN          NaN     NaN        NaN     2017-30    2017   
         21  NaN          NaN     NaN        NaN     2017-31    2017   
         22  NaN          NaN     NaN        NaN     2017-32    2017   
         23  NaN          NaN     NaN        NaN     2017-43    2017   
         24  NaN          NaN     NaN        NaN     2017-44    2017   
         25  NaN          NaN     NaN        NaN     2018-27    2018   
         26  NaN          NaN     NaN        NaN     2018-28    2018   

             WeekNum_y  
UPC                     
71090098 17         27  
         18         28  
         19         29  
         20         30  
         21         31  
         22         32  
         23         43  
         24         44  
         25         27  
         26         28

In [41]:
SalesSlicedf.isnull().sum()

UPC            17056
Gross Sales    17056
YEAR_x         17056
WeekNum_x      17056
YEARWeekNum        0
YEAR_y             0
WeekNum_y          0
dtype: int64

In [42]:
SalesSlicedf.drop(columns = ["UPC","YEAR_x", "WeekNum_x"], inplace = True) #drop the extra UPC columns that has null values
SalesSlicedf.reset_index(inplace = True) #reset index so true UPC is acolumns
SalesSlicedf.drop(columns = ["level_1"], inplace = True) #drop extra column

#fillnas Sales with 0
SalesSlicedf["Gross Sales"].fillna(0, inplace = True)


SalesSlicedf.rename(columns={"YEAR_y": "YEAR", "WeekNum_y": "WeekNum"}, inplace=True)

In [43]:
SalesSlicedf.shape

(40464, 5)

In [44]:
SalesSlicedf.columns

Index(['UPC', 'Gross Sales', 'YEARWeekNum', 'YEAR', 'WeekNum'], dtype='object')

In [45]:
TCWeekly.columns

Index(['UPC', 'YEAR', 'WeekNum', 'YEARWeekNum', 'Spend'], dtype='object')

In [46]:
len(TCWeekly.UPC.unique())

174

In [47]:
174*36 #post merge it should have 6264 rows

6264

In [48]:
#merge TCWeekly with all the YEARWeekNumdf
TCWeeklyFull = TCWeekly.groupby(["UPC"]).apply(mergeYWN)

In [49]:
TCWeeklyFull.shape #checked length correct

(6264, 7)

In [50]:
TCWeeklyFull.isnull().sum()

UPC            4365
YEAR_x         4365
WeekNum_x      4365
YEARWeekNum       0
Spend          4365
YEAR_y            0
WeekNum_y         0
dtype: int64

In [51]:
TCWeeklyFull.tail(3)

UPC  YEAR_x  WeekNum_x YEARWeekNum  Spend  YEAR_y  WeekNum_y
UPC                                                                          
836093000000 33  NaN     NaN        NaN     2018-42    NaN    2018         42
             34  NaN     NaN        NaN     2018-43    NaN    2018         43
             35  NaN     NaN        NaN     2018-44    NaN    2018         44

In [52]:
#do the same reorg as SalesSlicedf
TCWeeklyFull.drop(columns = ["UPC","YEAR_x", "WeekNum_x"], inplace = True) #drop the extra UPC columns that has null values
TCWeeklyFull.reset_index(inplace = True) #reset index so true UPC is acolumns
TCWeeklyFull.drop(columns = ["level_1"], inplace = True) #drop extra column
TCWeeklyFull["Spend"].fillna(0, inplace = True)

In [53]:
TCWeeklyFull.rename(columns={"YEAR_y": "YEAR", "WeekNum_y": "WeekNum"}, inplace=True)

In [54]:
TCWeeklyFull.tail()

,UPC,YEARWeekNum,Spend,YEAR,WeekNum
6259,836093000000,2018-31,0.0,2018,31
6260,836093000000,2018-41,0.0,2018,41
6261,836093000000,2018-42,0.0,2018,42
6262,836093000000,2018-43,0.0,2018,43
6263,836093000000,2018-44,0.0,2018,44


In [55]:
Tmerge = TCWeeklyFull.merge(SalesSlicedf, how ="inner", left_on = ["UPC", "YEARWeekNum"], right_on = ["UPC", "YEARWeekNum"])

In [56]:
Tmerge.shape

(5868, 8)

In [57]:
5868/163 #this si correct

36.0

In [58]:
Tmerge.head(2)

,UPC,YEARWeekNum,Spend,YEAR_x,WeekNum_x,Gross Sales,YEAR_y,WeekNum_y
0,12000002304,2018-35,0.0,2018,35,102.20,2018,35
1,12000002304,2018-36,8.0,2018,36,99.39,2018,36


In [60]:
Tmerge.to_csv("Tmerge.csv")

In [61]:
Tmerge.isnull().sum()

UPC            0
YEARWeekNum    0
Spend          0
YEAR_x         0
WeekNum_x      0
Gross Sales    0
YEAR_y         0
WeekNum_y      0
dtype: int64

In [62]:
Tmerge.drop(columns=["YEAR_x", "WeekNum_x"], inplace=True)

In [63]:
Tmerge.rename(columns={"YEAR_y": "YEAR", "WeekNum_y": "WeekNum"}, inplace=True)

In [64]:
Tmerge.head(3)

,UPC,YEARWeekNum,Spend,Gross Sales,YEAR,WeekNum
0,12000002304,2018-35,0.0,102.20,2018,35
1,12000002304,2018-36,8.0,99.39,2018,36
2,12000002304,2018-37,5.0,102.56,2018,37


In [65]:
def sidetoside(df):
    #each df is by individual UPC
    df2017 = df[df["YEAR"].isin([2017])].copy()
    df2017.rename(columns={"Spend": "2017Spend", "Gross Sales": "2017 Sales"}, inplace= True)
    df2018= df[df["YEAR"].isin([2018])].copy()
    df2018.rename(columns={"Spend": "2018Spend", "Gross Sales": "2018 Sales"}, inplace= True)
    result = df2017.merge(df2018, how="inner", left_on="WeekNum", right_on="WeekNum")
    return result

In [66]:
TS2S = Tmerge.groupby(["UPC"]).apply(sidetoside)

In [67]:
TS2S.head(10)

UPC_x YEARWeekNum_x  2017Spend  2017 Sales  YEAR_x  \
UPC                                                                       
12000002304 0  12000002304       2017-27        0.0         0.0    2017   
            1  12000002304       2017-28        0.0         0.0    2017   
            2  12000002304       2017-29        0.0         0.0    2017   
            3  12000002304       2017-30        0.0         0.0    2017   
            4  12000002304       2017-31        0.0         0.0    2017   
            5  12000002304       2017-32        0.0         0.0    2017   
            6  12000002304       2017-33        0.0         0.0    2017   
            7  12000002304       2017-34        0.0         0.0    2017   
            8  12000002304       2017-35        0.0         0.0    2017   
            9  12000002304       2017-36        0.0         0.0    2017   

               WeekNum        UPC_y YEARWeekNum_y  2018Spend  2018 Sales  \
UPC                                                                        
12000002304 0       27  12000002304       2018-27        0.0       98.82   
            1       28  12000002304       2018-28        0.0       93.72   
            2       29  12000002304       2018-29        0.0      127.80   
            3       30  12000002304       2018-30        0.0      101.30   
            4       31  12000002304       2018-31        0.0      110.30   
            5       32  12000002304       2018-32        0.0      127.09   
            6       33  12000002304       2018-33        0.0      112.38   
            7       34  12000002304       2018-34        0.0      112.62   
            8       35  12000002304       2018-35        0.0      102.20   
            9       36  12000002304       2018-36        8.0       99.39   

               YEAR_y  
UPC                    
12000002304 0    2018  
            1    2018  
            2    2018  
            3    2018  
            4    2018  
            5    2018  
            6    2018  
            7    2018  
            8    2018  
            9    2018

In [68]:
TS2S.reset_index(inplace=True)

In [69]:
TS2S.columns

Index(['UPC', 'level_1', 'UPC_x', 'YEARWeekNum_x', '2017Spend', '2017 Sales',
       'YEAR_x', 'WeekNum', 'UPC_y', 'YEARWeekNum_y', '2018Spend',
       '2018 Sales', 'YEAR_y'],
      dtype='object')

In [70]:
TS2S.drop(columns=['level_1', 'UPC_x','YEARWeekNum_x','YEAR_x','UPC_y', 'YEARWeekNum_y','YEAR_y'], inplace=True)

In [ ]:
#TS2S.to_csv("TS2S.csv")
#already have written the file

In [71]:
TS2StotSales = TS2S.groupby(["UPC"], as_index=False)["2018 Sales"].sum()

In [72]:
TS2StotSales.sort_values(by = ["2018 Sales"], ascending=False, inplace=True)

In [73]:
top20UPC = list(TS2StotSales["UPC"][:20])

In [74]:
TS2Stop = TS2S[TS2S["UPC"].isin(top20UPC)]

In [75]:
TS2Stop.shape

(360, 6)

In [77]:
#TS2Stop.to_csv("TS2Stop.csv")
#already written
#realized that 2017 had a lot of no sales, so if you want to check delta sales between 2017 and 2018 
#over the campaign period, it is not very telling

In [ ]:
##4/5/2019
## find the prior period where there is sales but no spend, and check for correlation btween sales and spend

In [81]:
#for the top 20 products, find the weeks there spend was 0 and when spend >0abs
Tmerge.head(3)

,UPC,YEARWeekNum,Spend,Gross Sales,YEAR,WeekNum
0,12000002304,2018-35,0.0,102.20,2018,35
1,12000002304,2018-36,8.0,99.39,2018,36
2,12000002304,2018-37,5.0,102.56,2018,37


In [83]:
Tmerge.WeekNum.unique()


array([35, 36, 37, 38, 39, 40, 41, 42, 43, 27, 28, 29, 30, 31, 32, 33, 34,
       44], dtype=int64)

In [84]:
Tmerge["Spend>0"] = (Tmerge["Spend"]>0).astype(int)

In [88]:
Tmerge2018 = Tmerge[Tmerge["YEAR"].isin([2018])].copy()


In [90]:
df2018 = Tmerge2018.groupby(["UPC", "Spend>0"], as_index=False).agg({"WeekNum":"count", "Spend": "sum", "Gross Sales": "sum"})

In [91]:
df2018.columns

Index(['UPC', 'Spend>0', 'WeekNum', 'Spend', 'Gross Sales'], dtype='object')

In [92]:
df2018.rename(columns={"WeekNum": "WeekNumCnt", "Spend": "totSpend", "Gross Sales": "totSales"}, inplace=True)

In [93]:
df2018.head(3)

,UPC,Spend>0,WeekNumCnt,totSpend,totSales
0,12000002304,0,12,0.0,1647.02
1,12000002304,1,6,33.0,794.36
2,12000012761,0,8,0.0,516.13


In [94]:
df2018["weeklySales"]=df2018["totSales"]/df2018["WeekNumCnt"]
df2018["weeklySpend"]=df2018["totSpend"]/df2018["WeekNumCnt"]

In [95]:
df2018.head(3)

,UPC,Spend>0,WeekNumCnt,totSpend,totSales,weeklySales,weeklySpend
0,12000002304,0,12,0.0,1647.02,137.251667,0.0
1,12000002304,1,6,33.0,794.36,132.393333,5.5
2,12000012761,0,8,0.0,516.13,64.516250,0.0


In [99]:
#first check if there are any no spend that did not have any sales
df2018[df2018["totSales"]==0]

,UPC,Spend>0,WeekNumCnt,totSpend,totSales,weeklySales,weeklySpend
253,30000318355,0,11,0.0,0.0,0.0,0.000000
254,30000318355,1,7,130.0,0.0,0.0,18.571429
264,30000322420,0,16,0.0,0.0,0.0,0.000000
265,30000322420,1,2,54.0,0.0,0.0,27.000000
283,30000562772,0,14,0.0,0.0,0.0,0.000000
284,30000562772,1,4,32.0,0.0,0.0,8.000000


In [100]:
df2018zeroSales = df2018[df2018["totSales"]==0].copy() #these will be removed from the table

In [152]:
df2018zeroSales

,UPC,Spend>0,WeekNumCnt,totSpend,totSales,weeklySales,weeklySpend
253,30000318355,0,11,0.0,0.0,0.0,0.000000
254,30000318355,1,7,130.0,0.0,0.0,18.571429
264,30000322420,0,16,0.0,0.0,0.0,0.000000
265,30000322420,1,2,54.0,0.0,0.0,27.000000
283,30000562772,0,14,0.0,0.0,0.0,0.000000
284,30000562772,1,4,32.0,0.0,0.0,8.000000


In [102]:
df2018.drop(df2018[df2018["totSales"]==0].index, inplace=True) #drop the no sales UPCs

In [111]:
checkmiss=df2018.groupby(["UPC"])["Spend>0"].count()<2

In [116]:
oneRowUPC = list(checkmiss[checkmiss].index)

In [118]:
df2018noZeroSales = df2018[df2018["UPC"].isin(oneRowUPC)].copy() 
#these only have weeks that have sales
#have to drop these too

In [119]:
df2018.drop(df2018[df2018["UPC"].isin(oneRowUPC)].index, inplace=True) #drop the no sales UPCs

In [120]:
checkmiss2=df2018.groupby(["UPC"])["Spend>0"].count()<2

In [122]:
checkmiss2[checkmiss2] #checked that all have been droped

Series([], Name: Spend>0, dtype: bool)

In [124]:
df2018.head(3)

,UPC,Spend>0,WeekNumCnt,totSpend,totSales,weeklySales,weeklySpend
0,12000002304,0,12,0.0,1647.02,137.251667,0.0
1,12000002304,1,6,33.0,794.36,132.393333,5.5
2,12000012761,0,8,0.0,516.13,64.516250,0.0


In [129]:
df2018.iloc[0]["weeklySales"]

137.25166666666664

In [130]:
df2018.sort_values(by=["UPC", "Spend>0"], inplace = True)

In [134]:
df2018.tail(4)

,UPC,Spend>0,WeekNumCnt,totSpend,totSales,weeklySales,weeklySpend
313,52000506402,0,6,0.0,1699.43,283.238333,0.000000
314,52000506402,1,12,840.0,4905.01,408.750833,70.000000
315,52000506488,0,6,0.0,1998.80,333.133333,0.000000
316,52000506488,1,12,538.0,5743.50,478.625000,44.833333


In [157]:
#split and merge
df2018left = df2018[df2018["Spend>0"]==0].copy()

In [158]:
df2018left.head(3)

,UPC,Spend>0,WeekNumCnt,totSpend,totSales,weeklySales,weeklySpend
0,12000002304,0,12,0.0,1647.02,137.251667,0.0
2,12000012761,0,8,0.0,516.13,64.516250,0.0
4,12000012792,0,11,0.0,2050.84,186.440000,0.0


In [159]:
df2018left.drop(columns=["Spend>0", "totSpend", "weeklySpend"], inplace = True)

In [162]:
df2018left.rename(columns={"WeekNumCnt": "noSpendWeekCnt", "totSales":"noSpendtotSales", 
                   "weeklySales": "noSpendweeklySales"}, inplace = True)

In [163]:
df2018left.head(3)

,UPC,noSpendWeekCnt,noSpendtotSales,noSpendweeklySales
0,12000002304,12,1647.02,137.251667
2,12000012761,8,516.13,64.516250
4,12000012792,11,2050.84,186.440000


In [164]:
df2018right = df2018[df2018["Spend>0"]==1].copy()

In [165]:
df2018right.head(3)

,UPC,Spend>0,WeekNumCnt,totSpend,totSales,weeklySales,weeklySpend
1,12000002304,1,6,33.0,794.36,132.393333,5.5
3,12000012761,1,10,57.0,1000.22,100.022000,5.7
5,12000012792,1,7,28.0,1079.30,154.185714,4.0


In [166]:
df2018right.rename(columns={"WeekNumCnt": "SpendWeekCnt", "totSales":"SpendtotSales", 
                   "weeklySales": "SpendweeklySales"}, inplace = True)
df2018right.drop(columns=["Spend>0"], inplace=True)

In [168]:
df2018right.head(3)

,UPC,SpendWeekCnt,totSpend,SpendtotSales,SpendweeklySales,weeklySpend
1,12000002304,6,33.0,794.36,132.393333,5.5
3,12000012761,10,57.0,1000.22,100.022000,5.7
5,12000012792,7,28.0,1079.30,154.185714,4.0


In [169]:
df2018left.shape

(151, 4)

In [170]:
df2018right.shape

(151, 6)

In [171]:
df2018M = df2018left.merge(df2018right, how = "outer", left_on=["UPC"], right_on =["UPC"])

In [172]:
df2018M.head(3)

,UPC,noSpendWeekCnt,noSpendtotSales,noSpendweeklySales,SpendWeekCnt,totSpend,SpendtotSales,SpendweeklySales,weeklySpend
0,12000002304,12,1647.02,137.251667,6,33.0,794.36,132.393333,5.5
1,12000012761,8,516.13,64.516250,10,57.0,1000.22,100.022000,5.7
2,12000012792,11,2050.84,186.440000,7,28.0,1079.30,154.185714,4.0


In [173]:
df2018M["delWeeklySales"] = df2018M["SpendweeklySales"]-df2018M["noSpendweeklySales"]
df2018M["weeklyROI"] = df2018M["delWeeklySales"]/df2018M["weeklySpend"]

In [174]:
df2018M.shape #check shape, should be 151 rows

(151, 11)

In [175]:
df2018M.to_csv("df2018M.csv")

In [142]:
# def calcdelSales(df):
#     #df is grouped by UPC
#     weeklyDelSales = df.iloc[1]["weeklySales"] - df.iloc[0]["weeklySales"]
#     weeklyROI = weeklyDelSales/(df.iloc[1]["weeklySpend"])
#     return weeklyDelSales, weeklyROI 